<h1>Fit a CNN on the pngs files</h1>

In [ ]:
# !pip install tensorflow

In [3]:
"""
    Fit a CNN per machine (on the pngs files in machine/png_train)
    from: https://keras.io/examples/vision/image_classification_from_scratch/
"""
import os 
import datetime
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# add parent folder to path 
from pathlib import Path
path = Path(os.getcwd())
str_parent = str(path.parent.absolute())
if not str_parent in sys.path:
    sys.path.append(str_parent)
import utils 
from SoundFile import SoundFile



In [5]:
image_size = (333, 216)
batch_size = 32
# epochs = 2
epochs = 8
data_augmentation = keras.Sequential(
    [
        # layers.experimental.preprocessing.RandomFlip("horizontal"),
        # layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [6]:

# Initialising the CNN v4: make model
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)    
    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256]:
    # for size in [128]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


In [7]:

model = make_model(input_shape=image_size + (3,))
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


In [8]:
# Part 2 - Fitting the CNN to the images
for folder_machine in utils.list_datasets: 
    use_folder = '../../_data_png_cnn1/' + folder_machine + '/png_train'
    print('Processing: ', use_folder, '...')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        use_folder,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        use_folder,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    model.fit(
        train_ds, epochs=epochs, validation_data=val_ds,
    )

    # save the model to disk for this machine ----------------------- ' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '_
    filename_classifier = '../../_classifiers_cnn1/cnn_' + folder_machine + '.h5'
    model.save(filename_classifier)
    print(folder_machine, ': model saved: ', filename_classifier)


Processing:  ../../_data_png_cnn1/valve/png_train ...
Found 3678 files belonging to 2 classes.
Using 2943 files for training.
Found 3678 files belonging to 2 classes.
Using 735 files for validation.
Epoch 1/8
92/92 [==============================] - 470s 5s/step - loss: 0.3534 - accuracy: 0.8838 - val_loss: 0.5988 - val_accuracy: 0.9020
Epoch 2/8
92/92 [==============================] - 463s 5s/step - loss: 0.3013 - accuracy: 0.8960 - val_loss: 0.5903 - val_accuracy: 0.9020
Epoch 3/8
92/92 [==============================] - 476s 5s/step - loss: 0.2715 - accuracy: 0.9076 - val_loss: 0.7089 - val_accuracy: 0.3986
Epoch 4/8
92/92 [==============================] - 484s 5s/step - loss: 0.2386 - accuracy: 0.9161 - val_loss: 1.0788 - val_accuracy: 0.0980
Epoch 5/8
92/92 [==============================] - 481s 5s/step - loss: 0.1920 - accuracy: 0.9378 - val_loss: 1.4786 - val_accuracy: 0.0980
Epoch 6/8
92/92 [==============================] - 480s 5s/step - loss: 0.1601 - accuracy: 0.9456 - v

/Users/david/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


valve : model saved:  ../../_classifiers_cnn1/cnn_valve.h5
Processing:  ../../_data_png_cnn1/slider/png_train ...
Found 3598 files belonging to 2 classes.
Using 2879 files for training.
Found 3598 files belonging to 2 classes.
Using 719 files for validation.
Epoch 1/8
90/90 [==============================] - 475s 5s/step - loss: 0.4303 - accuracy: 0.8440 - val_loss: 1.7714 - val_accuracy: 0.3352
Epoch 2/8
90/90 [==============================] - 451s 5s/step - loss: 0.3214 - accuracy: 0.8823 - val_loss: 0.4479 - val_accuracy: 0.7955
Epoch 3/8
90/90 [==============================] - 436s 5s/step - loss: 0.2319 - accuracy: 0.9107 - val_loss: 1.1172 - val_accuracy: 0.7858
Epoch 4/8
90/90 [==============================] - 433s 5s/step - loss: 0.1268 - accuracy: 0.9538 - val_loss: 2.5631 - val_accuracy: 0.7858
Epoch 5/8
90/90 [==============================] - 453s 5s/step - loss: 0.0765 - accuracy: 0.9764 - val_loss: 0.4556 - val_accuracy: 0.8053
Epoch 6/8
90/90 [========================